# E: State stabilization with SNAP gates and displacement gates

The use of feedback GRAPE applied to the Jaynes-
Cummings scenario allows us to discover strategies
extending the lifetime of a range of quantum states. How-
ever, for more complex quantum states such as kitten
states, the infidelity becomes significant after just a few
dissipative evolution steps in spite of the feedback [cf.
Fig. 6(c)]. This raises the question of whether the limited
quality of the stabilization is to be attributed to a failure
of our feedback-GRAPE learning algorithm to properly
explore the control-parameter landscape or, rather, to the
limited expressivity of the controls. With the goal of
addressing this question, we test our method on the state-
stabilization task using a more expressive control scheme.

In [1]:
# ruff: noqa
import sys, os
sys.path.append(os.path.abspath("./../feedback-grape"))
sys.path.append(os.path.abspath("./../"))

# ruff: noqa
from feedback_grape.fgrape import optimize_pulse, Decay, Gate, evaluate_on_longer_time
from feedback_grape.utils.operators import cosm, sinm, identity
from feedback_grape.utils.states import coherent
import jax.numpy as jnp
import jax
from feedback_grape.utils.operators import create, destroy
from feedback_grape.utils.fidelity import ket2dm
from library.utils.FgResult_to_dict import FgResult_to_dict
from tqdm import tqdm
import json

jax.config.update("jax_enable_x64", True)

## Initialize states

In [2]:
# Training parameters
N_training_iterations = 10000 # Number of training iterations
learning_rate = 0.01 # Learning rate
convergence_threshold = 1e-6 # Convergence threshold for early stopping

#num_time_steps : Number of time steps in the control pulse
#lut_depth : Depth of the lookup table for feedback
#reward_weights: Weights for the reward at each time step. Default only weights last timestep [0, 0, ... 0, 1]

samples = 4
samples_offset = 0
experiments = [ # (timesteps, lut_depth, reward_weights)
    #(1,1,[1]),
    #(2,1,[1,1]),
    #(2,2,[0,1]),
    #(2,2,[1,1]),
    #(3,3,[0,0,1]),
    (5,4,[1,1,1,1,1]),
    #(4,4,[0,0,0,1]),
    #(4,3,[0,0,0,1]),
    #(4,3,[0,0,1,1]),
    #(4,3,[1,1,1,1]),
    #(4,4,[1,1,1,1]),
]

# Physical parameters
N_cav = 30  # number of cavity modes
N_snap = 15

alpha = 2
psi_target = coherent(N_cav, alpha) + coherent(N_cav, -alpha)

# Normalize psi_target before constructing rho_target
psi_target = psi_target / jnp.linalg.norm(psi_target)

rho_target = ket2dm(psi_target)

for _, _, weights in experiments:
    for w in weights:
        assert type(w) == int and 0 <= w <= 9, "Weights must be integers between 0 and 9 to save as single character in filename."

## Initialize the parameterized Gates

In [3]:
def displacement_gate(alphas):
    """Displacement operator for a coherent state."""
    alpha_re, alpha_im = alphas
    alpha = alpha_re + 1j * alpha_im
    gate = jax.scipy.linalg.expm(
        alpha * create(N_cav) - alpha.conj() * destroy(N_cav)
    )
    return gate

def initialize_displacement_gate(key):
    return Gate(
        gate=displacement_gate,
        initial_params=jax.random.uniform(
            key,
            shape=(2,),
            minval=-jnp.pi / 2,
            maxval=jnp.pi / 2,
            dtype=jnp.float64,
        ),
        measurement_flag=False,
    )

def displacement_gate_dag(alphas):
    """Displacement operator for a coherent state."""
    return displacement_gate(alphas).conj().T

def initialize_displacement_gate_dag(key):
    return Gate(
        gate=displacement_gate_dag,
        initial_params=jax.random.uniform(
            key,
            shape=(2,),
            minval=-jnp.pi / 2,
            maxval=jnp.pi / 2,
            dtype=jnp.float64,
        ),
        measurement_flag=False,
    )

def snap_gate(phase_list):
    diags = jnp.ones(shape=(N_cav - len(phase_list)))
    exponentiated = jnp.exp(1j * jnp.array(phase_list))
    diags = jnp.concatenate((exponentiated, diags))
    return jnp.diag(diags)

def initialize_snap_gate(key):
    return Gate(
        gate=snap_gate,
        initial_params=jax.random.uniform(
            key,
            shape=(N_snap,),
            minval=-jnp.pi / 2,
            maxval=jnp.pi / 2,
            dtype=jnp.float64,
        ),
        measurement_flag=False,
    )

def povm_measure_operator(measurement_outcome, params):
    """
    POVM for the measurement of the cavity state.
    returns Mm ( NOT the POVM element Em = Mm_dag @ Mm ), given measurement_outcome m, gamma and delta
    """
    gamma, delta = params
    cav_operator = gamma * create(N_cav) @ destroy(N_cav) + delta * identity(N_cav) / 2
    angle = cav_operator
    meas_op = jnp.where(
        measurement_outcome == 1,
        cosm(angle),
        sinm(angle),
    )
    return meas_op

def initialize_povm_gate(key):
    return Gate(
        gate=povm_measure_operator,
        initial_params=jax.random.uniform(
            key,
            shape=(2,),  # 2 for gamma and delta
            minval=-jnp.pi / 2,
            maxval=jnp.pi / 2,
            dtype=jnp.float64,
        ),
        measurement_flag=True,
    )

decay_gate = Decay(c_ops=[jnp.sqrt(0.005) * destroy(N_cav)])

def initialize_system_params(key):
    keys = jax.random.split(key, 4)
    return [
        decay_gate,
        initialize_povm_gate(keys[0]),
        decay_gate,
        initialize_displacement_gate(keys[1]),
        initialize_snap_gate(keys[2]),
        initialize_displacement_gate_dag(keys[3])
    ]

## Initialize RNN of choice

In [4]:
import flax.linen as nn


# You can do whatever you want inside so long as you maintaing the hidden_size and output size shapes
class RNN(nn.Module):
    hidden_size: int  # number of features in the hidden state
    output_size: int  # number of features in the output (inferred from the number of parameters) just provide those attributes to the class

    @nn.compact
    def __call__(self, measurement, hidden_state):

        if measurement.ndim == 1:
            measurement = measurement.reshape(1, -1)

        ###############
        ### Free to change whatever you want below as long as hidden layers have size self.hidden_size
        ### and output layer has size self.output_size
        ###############

        gru_cell = nn.GRUCell(
            features=self.hidden_size,
            gate_fn=nn.sigmoid,
            activation_fn=nn.tanh,
        )
        self.make_rng('dropout')

        new_hidden_state, _ = gru_cell(hidden_state, measurement)
        new_hidden_state = nn.Dropout(rate=0.1, deterministic=False)(
            new_hidden_state
        )
        # this returns the povm_params after linear regression through the hidden state which contains
        # the information of the previous time steps and this is optimized to output best povm_params
        # new_hidden_state = nn.Dense(features=self.hidden_size)(new_hidden_state)
        new_hidden_state = nn.Dense(
            features=self.hidden_size,
            kernel_init=nn.initializers.glorot_uniform(),
        )(new_hidden_state)
        new_hidden_state = nn.relu(new_hidden_state)
        new_hidden_state = nn.Dense(
            features=self.hidden_size,
            kernel_init=nn.initializers.glorot_uniform(),
        )(new_hidden_state)
        new_hidden_state = nn.relu(new_hidden_state)
        output = nn.Dense(
            features=self.output_size,
            kernel_init=nn.initializers.glorot_uniform(),
            bias_init=nn.initializers.constant(0.1),
        )(new_hidden_state)
        output = nn.relu(output)

        ###############
        ### Do not change the return statement
        ###############

        return output[0], new_hidden_state

In [ ]:
for num_time_steps, lut_depth, reward_weights in experiments:
    for s in range(samples_offset, samples + samples_offset):
        print(f"Evaluating num_time_steps={num_time_steps}, lut_depth={lut_depth}, reward_weights={reward_weights}")
        print("Training LUT")

        system_params = initialize_system_params(jax.random.PRNGKey(s))
        weights_str = "".join([str(w) for w in reward_weights])

        # Train LUT
        result = optimize_pulse(
            U_0=rho_target,
            C_target=rho_target,
            system_params=system_params,
            num_time_steps=num_time_steps,
            lut_depth=lut_depth,
            reward_weights=reward_weights,
            mode="lookup",
            goal="fidelity",
            max_iter=N_training_iterations,
            convergence_threshold=convergence_threshold,
            learning_rate=learning_rate,
            evo_type="density",
            batch_size=16,
            progress=True,
        )

        with open(f"./optimized_architectures/lut_t={num_time_steps}_l={lut_depth}_s={s}_w={weights_str}.json", "w") as f:
            json.dump(FgResult_to_dict(result), f)

        result = evaluate_on_longer_time(
            U_0 = rho_target,
            C_target = rho_target,
            system_params = system_params,
            optimized_trainable_parameters = result.optimized_trainable_parameters,
            num_time_steps = 200,
            evo_type = "density",
            goal = "fidelity",
            eval_batch_size = 16, # Default is 10
            mode = "lookup",
            rnn = None,
            rnn_hidden_size = 30,
        )

        fidelities_lut = result.fidelity_each_timestep

        jnp.savez(f"./evaluation_results/lut_t={num_time_steps}_l={lut_depth}_s={s}_w={weights_str}.npz", fidelities_lut=jnp.array(fidelities_lut))

        # Check if RNN has to be trained (only if num_time_steps and weight are different)
        if os.path.exists(f"./optimized_architectures/rnn_t={num_time_steps}_s={s}_w={weights_str}.json"):
            print(f"RNN for t={num_time_steps} and weights={weights_str} already trained, skipping training.")
            continue
        """
        # Train RNN
        print("Training RNN")
        result = optimize_pulse(
            U_0=rho_target,
            C_target=rho_target,
            system_params=system_params,
            num_time_steps=num_time_steps,
            reward_weights=reward_weights,
            mode="nn",
            goal="fidelity",
            max_iter=N_training_iterations,
            convergence_threshold=convergence_threshold,
            learning_rate=learning_rate,
            evo_type="density",
            batch_size=16,
            rnn=RNN,
            rnn_hidden_size=30,
            progress=True,
        )

        with open(f"./optimized_architectures/rnn_t={num_time_steps}_s={s}_w={weights_str}.json", "w") as f:
            json.dump(FgResult_to_dict(result), f)

        result = evaluate_on_longer_time(
            U_0 = rho_target,
            C_target = rho_target,
            system_params = system_params,
            optimized_trainable_parameters = result.optimized_trainable_parameters,
            num_time_steps = 1000,
            evo_type = "density",
            goal = "fidelity",
            eval_batch_size = 16, # Default is 10
            mode = "nn",
            rnn = RNN,
            rnn_hidden_size = 30,
        )

        fidelities_rnn = result.fidelity_each_timestep

        jnp.savez(f"./evaluation_results/rnn_t={num_time_steps}_s={s}_w={weights_str}.npz", fidelities_rnn=jnp.array(fidelities_rnn))
        """

Evaluating num_time_steps=5, lut_depth=4, reward_weights=[1, 1, 1, 1, 1]
Training LUT


  0%|          | 0/1 [00:00<?, ?it/s]2025-11-12 17:14:29.086746: E external/xla/xla/service/slow_operation_alarm.cc:73] 
********************************
[Compiling module jit_step] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************
2025-11-12 17:14:45.619231: E external/xla/xla/service/slow_operation_alarm.cc:140] The operation took 2m16.537922s

********************************
[Compiling module jit_step] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************


Iteration 10, Loss (avg over 10): 0.269980, T=16s, eta=15134s
Iteration 20, Loss (avg over 10): 0.397390, T=32s, eta=15486s
Iteration 30, Loss (avg over 10): 0.491396, T=49s, eta=15765s
Iteration 40, Loss (avg over 10): 0.587060, T=64s, eta=15753s
Iteration 50, Loss (avg over 10): 0.644143, T=80s, eta=15778s
Iteration 60, Loss (avg over 10): 0.790601, T=97s, eta=15851s
Iteration 70, Loss (avg over 10): 0.748024, T=113s, eta=15903s
Iteration 80, Loss (avg over 10): 0.575093, T=129s, eta=15911s
Iteration 90, Loss (avg over 10): 0.586854, T=146s, eta=15995s
Iteration 100, Loss (avg over 10): 0.818738, T=163s, eta=16039s
Iteration 110, Loss (avg over 10): 0.727859, T=180s, eta=16054s
Iteration 120, Loss (avg over 10): 0.808879, T=196s, eta=16024s
Iteration 130, Loss (avg over 10): 0.786654, T=212s, eta=16008s
Iteration 140, Loss (avg over 10): 0.688877, T=228s, eta=16004s
Iteration 150, Loss (avg over 10): 0.508879, T=244s, eta=15979s
Iteration 160, Loss (avg over 10): 0.270110, T=260s, et

100%|██████████| 1/1 [4:36:13<00:00, 16573.48s/it]


Evaluating num_time_steps=5, lut_depth=4, reward_weights=[1, 1, 1, 1, 1]
Training LUT


  0%|          | 0/1 [00:00<?, ?it/s]

Iteration 10, Loss (avg over 10): 0.768254, T=19s, eta=18112s
Iteration 20, Loss (avg over 10): 1.008992, T=40s, eta=19029s
Iteration 30, Loss (avg over 10): 1.230231, T=60s, eta=19504s
Iteration 40, Loss (avg over 10): 1.383242, T=80s, eta=19631s
Iteration 50, Loss (avg over 10): 1.578208, T=101s, eta=19822s
Iteration 60, Loss (avg over 10): 1.693498, T=121s, eta=19824s
Iteration 70, Loss (avg over 10): 1.750624, T=140s, eta=19691s
Iteration 80, Loss (avg over 10): 1.788863, T=159s, eta=19574s
Iteration 90, Loss (avg over 10): 1.731958, T=178s, eta=19471s
Iteration 100, Loss (avg over 10): 1.806275, T=197s, eta=19407s
Iteration 110, Loss (avg over 10): 1.883236, T=217s, eta=19339s
Iteration 120, Loss (avg over 10): 1.955062, T=236s, eta=19284s
Iteration 130, Loss (avg over 10): 1.876863, T=255s, eta=19229s
Iteration 140, Loss (avg over 10): 1.834578, T=274s, eta=19174s
Iteration 150, Loss (avg over 10): 2.089791, T=293s, eta=19129s
Iteration 160, Loss (avg over 10): 2.194198, T=312s, 